<a href="https://colab.research.google.com/github/DeependraChaddha/Vlasov_Poisson_Solver/blob/main/vlasov_poisson_system_solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Script for Vlasov Poisson System Solver

In [ ]:
%%writefile vlasov_poisson_system.py
import os
import torch
import torch.nn as nn
import torch.optim
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from typing import Callable
import tqdm

class VlasovPoissonSolver:
    def __init__(self, nx: int, nv: int, nt: int, x_range: tuple, v_range: tuple, t_range: tuple, device: str):
        self.nx = nx
        self.nv = nv
        self.nt = nt
        self.x_range = x_range
        self.v_range = v_range
        self.t_range = t_range
        self.device = device if torch.cuda.is_available() else "cpu"
        self.model_checkpoint_path = None
        print("Instance of VlasovPoissonSolver created.")

    def make_grid(self):
        x = torch.linspace(self.x_range[0], self.x_range[1], self.nx).reshape(-1, 1)
        v = torch.linspace(self.v_range[0], self.v_range[1], self.nv).reshape(-1, 1)
        t = torch.linspace(self.t_range[0], self.t_range[1], self.nt).reshape(-1, 1)

        X, V, T = torch.meshgrid(x.squeeze(), v.squeeze(), t.squeeze(), indexing="ij")

        X = X.requires_grad_(True)
        V = V.requires_grad_(True)
        T = T.requires_grad_(True)

        self.X = X.to(self.device)
        self.V = V.to(self.device)
        self.T = T.to(self.device)
        return self.X, self.V, self.T

    def save_checkpoint(self, model, optimizer, loss, model_name, hyperparameters):
        checkpoint_dir = f"checkpoint_{model_name}_nx{self.nx}_nv{self.nv}_nt{self.nt}_epochs{hyperparameters['epochs']}"
        os.makedirs(checkpoint_dir, exist_ok=True)
        checkpoint_path = os.path.join(checkpoint_dir, "model_checkpoint.pkl")
        checkpoint = {
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "loss": loss,
            "hyperparameters": hyperparameters,
        }
        torch.save(checkpoint, checkpoint_path)
        self.model_checkpoint_path = checkpoint_path

    def train_step(self, model, loss_fn, optimizer, scheduler=None):
        # Forward Pass
        prediction = model(self.X, self.V, self.T, self.nx, self.nv, self.nt)

        # Compute Loss
        loss = loss_fn(model, self.X, self.V, self.T, self.nx, self.nv, self.nt)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if scheduler:
            scheduler.step(loss.item())

        return loss.item(), model.state_dict()

    def train(self, model, loss_fn, optimizer, epochs, model_name, get_loss_curve=False, scheduler=None):
        try:
            model = model.to(self.device)
            best_loss = float("inf")
            loss_values = []

            for epoch in tqdm.tqdm(range(epochs)):
                loss, model_params = self.train_step(model, loss_fn, optimizer, scheduler)
                loss_values.append(loss)

                if loss < best_loss:
                    best_loss = loss
                    self.save_checkpoint(model, optimizer, loss, model_name, {"epochs": epochs})

            print(f"Best Loss Achieved: {best_loss}")

            if get_loss_curve:
                plt.figure(figsize=(8, 5))
                plt.plot(range(1, epochs + 1), loss_values, marker="o", linestyle="-", color="r", label="Loss")
                plt.xlabel("Epoch")
                plt.ylabel("Loss")
                plt.title("Loss Curve Over Epochs")
                plt.legend()
                plt.grid()
                loss_curve_path = os.path.join(self.checkpoint_dir, "loss_curve.png")
                plt.savefig(loss_curve_path)
                print(f"Loss curve saved at {loss_curve_path}")
                plt.show()

            return self.model_checkpoint_path
        except AttributeError:
            print("Make grid first by calling make_grid()")

    def animate_final_prediction(self, model_class, model_checkpoint_path=None):
        try:
            if model_checkpoint_path is None:
                if self.model_checkpoint_path is None:
                    raise AttributeError("Model checkpoint not found. Train the model first.")
                model_checkpoint_path = self.model_checkpoint_path

            checkpoint = torch.load(model_checkpoint_path, map_location=self.device)
            model = model_class().to(self.device)
            model.load_state_dict(checkpoint["model_state_dict"])
            model.eval()

            fig, ax = plt.subplots(figsize=(8, 6))

            with torch.no_grad():
                F_all = model(self.X, self.V, self.T, self.nx, self.nv, self.nt)  # Compute for all time steps

            def update(frame):
                ax.clear()
                ax.imshow(F_all[:, :, frame].cpu().numpy(), extent=[self.x_range[0], self.x_range[1], self.v_range[0], self.v_range[1]], origin="lower", cmap="viridis")
                ax.set_xlabel("x")
                ax.set_ylabel("v")
                ax.set_title(f"Distribution Function at Time {frame}")

            ani = FuncAnimation(fig, update, frames=self.nt, repeat=False)

            os.makedirs(self.checkpoint_dir, exist_ok=True)
            animation_path = os.path.join(self.checkpoint_dir, "vlasov_distribution.gif")
            ani.save(animation_path, writer=PillowWriter(fps=10))

            self.model_checkpoint_path = animation_path
            print(f"Animation saved at: {animation_path}")

            plt.close(fig)
            plt.show()
        except AttributeError as e:
            print(e)


#Using above script

In [ ]:
import time_dependent_heat_equation  # Rename this module to match your modified version
from time_dependent_heat_equation import TimeDependentHeatEquationSolver
import torch
import torch.nn as nn

class DeepRitzVlasovPoisson(nn.Module):
    def __init__(self, input_dim=3, hidden_dim=128, num_layers=8, use_fourier=True):
        super().__init__()
        self.use_fourier = use_fourier

        self.input_layer = nn.Linear(hidden_dim if use_fourier else input_dim, hidden_dim)
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_dim, hidden_dim) for _ in range(num_layers - 1)])
        self.output_layer = nn.Linear(hidden_dim, 1)
        self.activation = nn.SiLU()

        if use_fourier:
            self.freq_embedding = nn.Linear(input_dim, hidden_dim)
            torch.nn.init.normal_(self.freq_embedding.weight, mean=0, std=1)

    def forward(self, X, V, T, nx, nv, nt):
        X, V, T = X.reshape(-1, 1), V.reshape(-1, 1), T.reshape(-1, 1)
        inputs = torch.cat((X, V, T), dim=1)

        if self.use_fourier:
            inputs = torch.sin(self.freq_embedding(inputs))

        x = self.activation(self.input_layer(inputs))  # Now input matches dimensions

        for layer in self.hidden_layers:
            residual = x
            x = self.activation(layer(x)) + residual

        return self.output_layer(x).view(nx, nv, nt)  # Ensure correct shape

model = DeepRitzVlasovPoisson()

'''Defining Loss function'''
def loss_fn(model: nn.Module, x: torch.Tensor, v: torch.Tensor, t: torch.Tensor,
            nx: int, nv: int, nt: int,
            boundary_loss_weight: float = 2.0,
            initial_loss_weight: float = 2.0):

    prediction = model(x, v, t, nx, nv, nt)

    # Compute first-order gradients
    f_x = torch.autograd.grad(prediction, x, torch.ones_like(prediction), create_graph=True)[0]
    f_v = torch.autograd.grad(prediction, v, torch.ones_like(prediction), create_graph=True)[0]
    f_t = torch.autograd.grad(prediction, t, torch.ones_like(prediction), create_graph=True)[0]

    # Compute the electric field by integrating f(x, v, t) over v
    rho = torch.trapz(prediction, v, dim=1)  # Charge density (integrating over v)
    E_x = torch.autograd.grad(rho, x, torch.ones_like(rho), create_graph=True)[0]  # ∂E/∂x = 1 - ρ

    # Vlasov equation loss: ∂f/∂t + v∂f/∂x + E∂f/∂v = 0
    pde_loss = torch.mean((f_t + v * f_x + E_x * f_v) ** 2)

    # Boundary conditions (periodic in x, decay in v)
    boundary_loss = (
        (prediction[0, :, :] - prediction[-1, :, :])**2 +  # Periodic in x
        (prediction[:, 0, :] - torch.zeros_like(prediction[:, 0, :]))**2 +  # Decay in v
        (prediction[:, -1, :] - torch.zeros_like(prediction[:, -1, :]))**2
    ).mean()

    # Initial condition loss (e.g., Maxwellian or given distribution)
    initial_condition = torch.exp(-v[:, :, 0]**2) * (1 + 0.1 * torch.cos(2 * torch.pi * x[:, :, 0]))
    initial_condition_loss = torch.mean((prediction[:, :, 0] - initial_condition) ** 2)

    total_loss = pde_loss + (boundary_loss_weight * boundary_loss) + (initial_loss_weight * initial_condition_loss)
    return total_loss

'''Set Optimizer'''
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

'''Add Scheduler'''
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

'''Make solver instance'''
current_solver = TimeDependentHeatEquationSolver(nx=30, ny=30, nt=30, x_range=(0, 1), y_range=(-5, 5), t_range=(0, 1), device='cpu')

'''Make Grid'''
X, V, T = current_solver.make_grid()

'''Train'''
Z = current_solver.train(model=model,
                         loss_fn=loss_fn,
                         optimizer=optimizer,
                         epochs=5,
                         model_name='DeepRitz_VlasovPoisson',
                         get_loss_curve=True,
                         scheduler=scheduler)
